## Music Genre Detector

### Importing Libraries

In [1]:
import os
import pickle
import random 
import operator
import math
from collections import defaultdict
import random

In [2]:
import scipy.io.wavfile as wav
from python_speech_features import mfcc
import numpy as np

### Feature Extraction

In [10]:
# Extract features from music files and dump them into .dat file
f= open("my.dat" ,'wb')
for folder in next(os.walk("./genres"))[1]:
    for file in os.listdir(f"./genres/{folder}"): 
        (rate,sig) = wav.read(f"./genres/{folder}/{file}")
        mfcc_feat = mfcc(sig,rate ,winlen=0.020, appendEnergy = False)
        covariance = np.cov(np.matrix.transpose(mfcc_feat))
        mean_matrix = mfcc_feat.mean(0)
        feature = (mean_matrix , covariance , i)
        pickle.dump(feature , f)
f.close()

###  Load dataset and split into train and test

In [3]:
dataset = []

def loadDataset(filename, split, trSet, tsSet):
    with open(filename, 'rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break
        for x in range(len(dataset)):
            if(random.random() < split):
                trSet.append(dataset[x])
            else:
                tsSet.append(dataset[x])

training_Set = []
testing_Set = []
loadDataset("my.dat", 0.66, training_Set, testing_Set)
print(len(training_Set),len(testing_Set))

645 354


### Distance

In [4]:
def distance(instance1, instance2, k):
    distance =0 
    mm1 = instance1[0] 
    cm1 = instance1[1]
    mm2 = instance2[0]
    cm2 = instance2[1]
    distance = np.trace(np.dot(np.linalg.inv(cm2), cm1)) 
    distance+=(np.dot(np.dot((mm2-mm1).transpose() , np.linalg.inv(cm2)) , mm2-mm1 )) 
    distance+= np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance-= k
    return distance

### Get Neighbors

In [5]:
# Function to get the distance between feature vectors and find neighbors
def getNeighbors(training_Set, instance, k):
    distances = []
    for x in range (0,len(training_Set)):
        dist = distance(training_Set[x], instance, k)+ distance(instance, training_Set[x], k)
        distances.append((training_Set[x][2], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(0,k):
        neighbors.append(distances[x][0])
    return neighbors

### Nearest Class

In [6]:
# function to identify class of the instance
def nearestClass(neighbors):
    classVote = {}

    for x in range(0,len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response]+=1 
        else:
            classVote[response]=1

    sorter = sorted(classVote.items(), key = operator.itemgetter(1), reverse=True)
    return sorter[0][0]

### Get Accuracy

In [7]:
# Function to evaluate model
def getAccuracy(testSet, predictions):
    correct = 0 
    for x in range (len(testSet)):
        if testSet[x][-1]==predictions[x]:
            correct+=1
    return (1.0*correct)/len(testSet)

### Make Predictions

In [10]:
leng = len(testing_Set)
predictions = []
for x in range (leng):
    predictions.append(nearestClass(getNeighbors(training_Set ,testing_Set[x] , 5))) 

accuracy1 = getAccuracy(testing_Set , predictions)
print(accuracy1)

KeyboardInterrupt: 

###  __ Main __

#### Genres

In [12]:
result = defaultdict(list)
i = 1
for folder in next(os.walk("./genres"))[1]:
    result[i] = folder
    i+=1
print(result)

defaultdict(<class 'list'>, {1: 'blues', 2: 'classical', 3: 'country', 4: 'disco', 5: 'hiphop', 6: 'jazz', 7: 'metal', 8: 'pop', 9: 'reggae', 10: 'rock'})


#### Make Prediction

In [15]:
(rate,sig) = wav.read(f"./testing_file/test2.wav")
mfcc_feat = mfcc(sig,rate ,winlen=0.020, appendEnergy = False)
covariance = np.cov(np.matrix.transpose(mfcc_feat))
mean_matrix = mfcc_feat.mean(0)
feature = (mean_matrix , covariance , i)
pred=nearestClass(getNeighbors(dataset ,feature , 5))

#### Print Result

In [16]:
print(result[pred])

blues
